In [1]:
#Imports
import os
import glob as glob
from pathlib import Path

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd

from matplotlib import pyplot as plt
import seaborn as sns
#Calculate Correlation
from scipy.stats import pearsonr
import os
import glob
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import platform

from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti

In [2]:
analysis = 'response_topology/voxelwise_mixed_effects/age-stim-interaction_pd-vs-ad'
clinical_data_path = r'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/clinical_analyses/ses-01/sub-all/ad_pd_mixed_data/ad_pd_full_data.csv'
# clin_path = 'path to clinical values'
out_dir = os.path.join(os.path.dirname(clinical_data_path), f'{analysis}')
print('Will save to: ', out_dir)
save = True
if os.path.exists(out_dir):
    pass
else:
    os.makedirs(out_dir)

Will save to:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/clinical_analyses/ses-01/sub-all/ad_pd_mixed_data/response_topology/voxelwise_mixed_effects/age-stim-interaction_pd-vs-ad


# Import Clinical Data

In [3]:
#----------------------------------------------------------------user input above----------------------------------------------------------------
data_df = pd.read_csv(clinical_data_path)
    
# #Remove subjects
outlier_index=[11, 47, 48, 49]
data_df = data_df.drop(index=outlier_index)
data_df.reset_index(drop=True, inplace=True)

data_df

,"Patient # CDR, ADAS",Age,% Change from baseline (ADAS-Cog11),Subiculum Connectivity,Subiculum Grey Matter,Subiculum White Matter,Subiculum CSF,Subiculum Total,Disease,Standardized Age,Standardized Percent Improvement,Standardized Subiculum Connectivity,Standardized Subiculum Grey Matter,Standardized Subiculum White Matter,Standardized Subiculum CSF,Standardized Subiculum Total,One Hot Disease
0,101,62,-21.428571,56.864683,1.646994,0.510111,2.975675,1.280978,Alzheimer,0.289341,0.289341,-1.179578,1.970962,0.663981,1.773701,1.789087,1
1,102,77,-36.363636,52.970984,-0.865158,-1.615736,0.740780,-1.560273,Alzheimer,-0.016378,-0.016378,-1.635523,-0.125938,-1.611307,-0.272553,-1.093456,1
2,103,76,-78.947368,62.459631,0.319460,-0.796399,0.532453,-0.341032,Alzheimer,-0.888064,-0.888064,-0.524419,0.862866,-0.734373,-0.463295,0.143505,1
3,104,65,-129.411765,59.611631,-0.440643,-0.159752,0.405263,-0.358042,Alzheimer,-1.921065,-1.921065,-0.857915,0.228406,-0.052972,-0.579749,0.126248,1
4,105,50,-10.526316,57.928350,-0.328427,-0.229875,-0.988805,-0.339817,Alzheimer,0.512509,0.512509,-1.055024,0.322072,-0.128024,-1.856148,0.144737,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,30,58,-1.388889,18.674670,6.973009,2.352260,6.973009,0.872723,Parkinson,-0.005890,-0.005890,-0.628666,0.291196,1.089313,0.291196,-0.792728,0
68,31,64,1.449275,15.353030,-8.205764,5.376292,-8.205764,8.959061,Parkinson,1.034438,1.034438,-1.099623,-3.303201,2.909124,-3.303201,2.775495,0
69,33,60,-1.398601,15.050219,9.086053,0.302823,9.086053,1.435556,Parkinson,-0.009451,-0.009451,-1.142557,0.791574,-0.144002,0.791574,-0.544369,0
70,36,52,-2.797203,17.382020,8.416993,1.409619,8.416993,-0.365509,Parkinson,-0.522107,-0.522107,-0.811944,0.633137,0.522049,0.633137,-1.339117,0


Get More Clinical Data As Needed

In [4]:
subject_column = 'Patient # CDR, ADAS'
clinical_information_column = ['Standardized Age', 'One Hot Disease']
outcome_column = 'Standardized Percent Improvement'

#----------------------------------------------------------------DO NOT MODIFY--------------------------------------------------------
clinical_df_1 = pd.DataFrame()
clinical_df_1 = data_df[clinical_information_column]

clinical_df_1['subject_id'] = data_df.loc[:, [subject_column]]
clinical_df_1['outcome'] = data_df.loc[:, [outcome_column]]

# clinical_df_1['subject_id'] = [id.split('_')[0] for id in data_df[subject_column].to_list()] 
clinical_df_1.set_index('subject_id', inplace=True)

# Convert the 'subject_id' column to strings for each DataFrame
clinical_df_1.index = clinical_df_1.index.astype(str)
display(clinical_df_1)

,Standardized Age,One Hot Disease,outcome
subject_id,,,
101,0.289341,1,0.289341
102,-0.016378,1,-0.016378
103,-0.888064,1,-0.888064
104,-1.921065,1,-1.921065
105,0.512509,1,0.512509
...,...,...,...
30,-0.005890,0,-0.005890
31,1.034438,0,1.034438
33,-0.009451,0,-0.009451


Import Imaging Data

In [5]:
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity'
matrix = import_matrices_from_folder(matrix_path, file_pattern='*_T*.nii*')

#Standardize matrix via z score
prepped_matrix = pd.DataFrame()
from calvin_utils.statistical_utils.z_score_matrix import z_score_matrix
for column in matrix.columns:
    #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
    print(column)
    new_name = int(column.split('_')[0]) #.split('T')[1]) #<----------------------------------THIS IS A BUG-CREATOR. MUST BE TAILORED TO PATIENT DATA
    #Standardize column by column
    prepped_matrix[new_name] = z_score_matrix(matrix[column])
    #Set values outside brain back to zero
    prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
prepped_matrix = prepped_matrix.transpose()
prepped_matrix['subject_id'] = [str(col).split('_')[0] for col in prepped_matrix.index]
prepped_matrix.set_index('subject_id', inplace=True)
neuroimaging_df_1 = prepped_matrix
neuroimaging_df_1.index = neuroimaging_df_1.index.astype(str)
#Display results
display(neuroimaging_df_1)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/connectivity_data/vta_published_t_connectivity/*_T*.nii*
116_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  195.56159244766604 (902629,)
post z score max:  36.16030539421581 (902629,)
1203_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  162.6742436890163 (902629,)
post z score max:  28.965697798110018 (902629,)
139_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  193.79302616660425 (902629,)
post z score max:  33.171095565585055 (902629,)
147_vat_seed_compound_fMRI_efield_func_seed_T.nii
--------------------------------Performing z_score--------------------------------
pre z score max:  209.41335490

,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject_id,,,,,,,,,,,,,,,,,,,,,
116,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
147,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
150,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
122,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Import More Imaging Data

In [6]:
#Import Another Dataframe if desired
from calvin_utils.file_utils.import_matrices import import_matrices_from_folder
from calvin_utils.nifti_utils.generate_nifti import nifti_from_matrix
from nimlab import datasets as nimds
import numpy as np
from nilearn import image, plotting, maskers

#get conectivity values of interest
matrix_path = '/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/vtasCompoundMNI152/subgroup_with_cognitive_scores/cog_decline_patient_connectivity'
neuroimaging_df_2 = import_matrices_from_folder(matrix_path, file_pattern='*T.nii*')
performed_z_score = False 
#----------------------------------------------------------------DO NOT MODIFY!----------------------------------------------------------------
#Standardize matrix via z score
# neuroimaging_df_2 = pd.DataFrame()
# from calvin_utils.z_score_matrix import z_score_matrix
# if performed_z_score:
#     for column in matrix.columns:
#         #Rename while we're at it. Needs to be an integer, as this is the type of patient ID in the clinical data
#         new_name = int(column.split('_')[0])
#         #Standardize column by column
#         prepped_matrix[new_name] = z_score_matrix(matrix[column])
#         #Set values outside brain back to zero
#         prepped_matrix[new_name] = np.where(matrix[column] == 0, 0, prepped_matrix[new_name])

#Set patients to those in the clinical data dataframe
# prepped_matrix = prepped_matrix.loc[:, data_df.index]

neuroimaging_df_2 = neuroimaging_df_2.transpose()
neuroimaging_df_2['subject_id'] = [col.split('_')[0].split('MDST')[1] for col in neuroimaging_df_2.index]
neuroimaging_df_2.set_index('subject_id', inplace=True)

neuroimaging_df_2.index = neuroimaging_df_2.index.astype(str)


#Display results
display(neuroimaging_df_2)

I will search:  /Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_PD_DBS_STN_WURZBURG/vtasCompoundMNI152/subgroup_with_cognitive_scores/cog_decline_patient_connectivity/*T.nii*


,0,1,2,3,4,5,6,7,8,9,...,902619,902620,902621,902622,902623,902624,902625,902626,902627,902628
subject_id,,,,,,,,,,,,,,,,,,,,,
43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# perform masking

In [7]:
#Mask Dataframes
mask_path = None #'/Users/cu135/Dropbox (Partners HealthCare)/resources/datasets/BIDS_AD_DBS_FORNIX/response_topology/voxelwise_glm/stim_by_age/palm_results/palm_statistic_topology/generated_nifti.nii' #None #
masking_df = None # # 
#----------------------------------------------------------------DO NOT MODIFY
from calvin_utils.nifti_utils.matrix_utilities import mask_matrix
# def mask_matrix(df_1, mask_path=None, mask_threshold=0.2, mask_by='rows', dataframe_to_mask_by=None):

neuroimaging_df_1 = mask_matrix(neuroimaging_df_1, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)
neuroimaging_df_2 = mask_matrix(neuroimaging_df_2, mask_path=mask_path, mask_threshold=0, mask_by='columns', dataframe_to_mask_by=masking_df)

Dataframes have been masked such that their shapes are:  (50, 225222)
Dataframes have been masked such that their shapes are:  (32, 225222)


## Join Imaging Data to the Clinical Data

In [8]:
from calvin_utils.file_utils.dataframe_utilities import preprocess_colnames_for_regression
# Stack neuroimaging dataframes
stacked_neuroimaging_df = pd.concat([neuroimaging_df_1, neuroimaging_df_2], axis=0)
stacked_neuroimaging_df
# # Perform inner join with clinical_df
merged_df = preprocess_colnames_for_regression(clinical_df_1.join(stacked_neuroimaging_df, how='inner'))

merged_df.iloc[:,0:100].to_csv(os.path.join('/Users/cu135/Downloads', 'csv_for_gpt.csv'))

# Run The Mixed Effects Analysis

In [116]:
from calvin_utils.file_utils.print_suppression import HiddenPrints

def voxelwise_mixed_effects_regression_updated(data_df, formula_template, voxel_columns, random_effects_column, model_type='linear'):
    """
    Perform mixed-effects regression voxelwise based on the provided formula template.
    
    Parameters:
        data_df (pd.DataFrame): DataFrame containing outcome, voxel values, clinical covariates, and other variables with subjects in rows.
        formula_template (str): A string template for the regression formula with 'voxel' as a placeholder for voxel columns.
        voxel_columns (list): List of voxel column names in data_df.
        model_type (str, default='linear'): Specifies the type of regression model to use ('linear' or 'logistic').
    
    Returns:
        results_df (pd.DataFrame): DataFrame containing p-values, coefficient values, t-values for each voxel,
                                   along with the coefficient, t-value, and p-value for each predictor.
    """
    
    all_results = []

    # Loop through each voxel column and fit the model
    for voxel in tqdm(voxel_columns):
        formula = formula_template.replace('voxel', voxel)
        
        try:
            if model_type == 'linear':
                with HiddenPrints():
                    model = sm.MixedLM.from_formula(formula, data=data_df, groups=data_df[random_effects_column]).fit()
            elif model_type == 'logistic':
                raise NotImplementedError("Mixed-effects logistic regression is not directly supported by statsmodels.")
            else:
                raise ValueError(f"Unsupported model_type: {model_type}")

            # Extract results for each predictor and store in dictionary
            results = {'voxel': voxel}

            for predictor in model.params.index:
                results[f'{predictor}_coeff'] = model.params[predictor]
                results[f'{predictor}_t_value'] = model.tvalues[predictor]
                results[f'{predictor}_p_value'] = model.pvalues[predictor]

        except Exception as e:
            if str(e) == "Singular matrix":
                results = {'voxel': voxel}
                for predictor in formula.split("~")[1].split("+"):
                    predictor = predictor.strip()
                    results[f'{predictor}_coeff'] = np.nan
                    results[f'{predictor}_t_value'] = np.nan
                    results[f'{predictor}_p_value'] = np.nan
            else:
                raise e  # if it's another error, we raise it

        all_results.append(results)

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(all_results)
    
    return results_df


In [117]:
formula = 'outcome ~ Standardized_Age + voxel  + Standardized_Age:voxel'
try:
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        results_updated = voxelwise_mixed_effects_regression_updated(merged_df, formula, merged_df.columns[merged_df.columns.get_loc('outcome')+1:], 'One_Hot_Disease', model_type='linear')
except Exception as e:
    error_message = str(e)

error_message if 'error_message' in locals() else "Successfully completed without errors."

  0%|          | 0/225222 [00:00<?, ?it/s]

100%|██████████| 225222/225222 [9:42:50<00:00,  6.44it/s]      


: 

: 

Test

In [9]:
from calvin_utils.file_utils.print_suppression import HiddenPrints
import gc
import os
from tqdm import tqdm
import statsmodels.regression.mixed_linear_model as sm

def voxelwise_mixed_effects_regression_updated(data_df, formula_template, voxel_columns, random_effects_column, model_type='linear', batch_size=5000, checkpoint_path='checkpoint.parquet'):
    """
    Perform mixed-effects regression voxelwise based on the provided formula template.
    
    Parameters:
        data_df (pd.DataFrame): DataFrame containing outcome, voxel values, clinical covariates, and other variables with subjects in rows.
        formula_template (str): A string template for the regression formula with 'voxel' as a placeholder for voxel columns.
        voxel_columns (list): List of voxel column names in data_df.
        random_effects_column (str): The column in data_df to be used for random effects.
        model_type (str, default='linear'): Specifies the type of regression model to use ('linear' or 'logistic').
        batch_size (int, default=5000): Number of voxels to process before saving a checkpoint.
        checkpoint_path (str, default='checkpoint.parquet'): Path to save the intermediate results as a checkpoint.

    Returns:
        results_df (pd.DataFrame): DataFrame containing p-values, coefficient values, t-values for each voxel,
                                   along with the coefficient, t-value, and p-value for each predictor.
    """
    
    all_results = []

    # Check if checkpoint exists, and if so, load it and continue from the last processed voxel
    if os.path.exists(checkpoint_path):
        all_results = pd.read_parquet(checkpoint_path).to_dict('records')
        start_idx = len(all_results)
    else:
        start_idx = 0

    # Loop through each voxel column and fit the model
    for idx, voxel in enumerate(tqdm(voxel_columns[start_idx:])):
        formula = formula_template.replace('voxel', voxel)
        
        try:
            if model_type == 'linear':
                with HiddenPrints():
                    model = sm.MixedLM.from_formula(formula, data=data_df, groups=data_df[random_effects_column]).fit()
            elif model_type == 'logistic':
                raise NotImplementedError("Mixed-effects logistic regression is not directly supported by statsmodels.")
            else:
                raise ValueError(f"Unsupported model_type: {model_type}")

            # Extract results for each predictor and store in dictionary
            results = {'voxel': voxel}

            for predictor in model.params.index:
                results[f'{predictor}_coeff'] = model.params[predictor]
                results[f'{predictor}_t_value'] = model.tvalues[predictor]
                results[f'{predictor}_p_value'] = model.pvalues[predictor]

        except Exception as e:
            if str(e) == "Singular matrix":
                results = {'voxel': voxel}
                for predictor in formula.split("~")[1].split("+"):
                    predictor = predictor.strip()
                    results[f'{predictor}_coeff'] = np.nan
                    results[f'{predictor}_t_value'] = np.nan
                    results[f'{predictor}_p_value'] = np.nan
            else:
                raise e  # if it's another error, we raise it

        all_results.append(results)

        # Save a checkpoint if processed voxel count is a multiple of batch_size
        if (idx + 1) % batch_size == 0:
            pd.DataFrame(all_results).to_parquet(checkpoint_path)
            gc.collect()

    # Convert the results to a DataFrame
    results_df = pd.DataFrame(all_results)
    
    return results_df


In [10]:
merged_df.columns

Index(['Standardized_Age', 'One_Hot_Disease', 'outcome', 'var_93491',
       'var_93492', 'var_93493', 'var_93582', 'var_93583', 'var_93672',
       'var_93673',
       ...
       'var_798109', 'var_798110', 'var_798190', 'var_798191', 'var_798192',
       'var_798193', 'var_798281', 'var_798282', 'var_798283', 'var_798373'],
      dtype='object', length=225225)

In [11]:
formula = 'outcome ~ Standardized_Age + voxel  + Standardized_Age:voxel'
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    voxelwise_mixed_effects_regression_updated(merged_df, 
                                               formula_template=formula, 
                                               voxel_columns=merged_df.columns[merged_df.columns.get_loc('outcome')+1:], 
                                               random_effects_column = 'One_Hot_Disease', 
                                               model_type='linear', 
                                               batch_size=100000, 
                                               checkpoint_path='checkpoint.parquet')

: 

: 

Recompose the Batches

In [ ]:
import pandas as pd

# List of checkpoint paths for each batch
checkpoint_paths = ['checkpoint_batch1.pkl', 'checkpoint_batch2.pkl', 'checkpoint_batch3.pkl', 'checkpoint_batch4.pkl']

# Load each checkpoint and store in a list
dfs = [pd.read_pickle(checkpoint) for checkpoint in checkpoint_paths]

# Concatenate all DataFrames vertically
combined_df = pd.concat(dfs, axis=0)

In [ ]:
results_updated

# Save the Results to Niftis

In [ ]:
import os
from calvin_utils.nifti_utils.generate_nifti import view_and_save_nifti
from calvin_utils.nifti_utils.matrix_utils import unmask_matrix

def save_results_as_nifti(results_df, out_dir, mask_path=None, mask_threshold=0.2, unmask_by='rows', dataframe_to_unmask_by=None):
    """
    Save each column in the results DataFrame as a NIFTI file.
    
    Parameters:
        results_df (pd.DataFrame): DataFrame containing various statistical measures for each voxel.
        out_dir (str): Directory where NIFTI files should be saved.
        mask_path (str, optional): Path to the NIFTI mask file to use for unmasking.
        mask_threshold (float, optional): Mask threshold for unmasking.
        unmask_by (str, optional): Direction for unmasking ('rows' or 'columns').
        dataframe_to_unmask_by (pd.DataFrame, optional): DataFrame to use for unmasking.
        
    Returns:
        None
    """
    
    # Ensure the output directory exists
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    
    # Iterate over every column in results_df and generate NIFTI files
    for colname in results_df.columns:
        # Unmask the matrix
        unmasked_df = unmask_matrix(results_df[colname], mask_path=mask_path, mask_threshold=mask_threshold,
                                    unmask_by=unmask_by, dataframe_to_unmask_by=dataframe_to_unmask_by)
        
        # Save the unmasked matrix as a NIFTI file
        view_and_save_nifti(unmasked_df, out_dir, output_name=colname)

In [ ]:
save_results_as_nifti(results_df, out_dir, mask_path=None, mask_threshold=0.2, unmask_by='rows', dataframe_to_unmask_by=None)

Enjoy

-Calvin